### Este notebook é utilizado para o desenvolvimento do script pyspark da camada bronze

In [1]:
%%sh
pip install delta-spark==2.1.1
pip3 install delta-spark==2.1.1

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=571e4611aef05e36948838ab7819ae42056ebb1a6dd20113c97f6f0e5f41d5c5
  Stored in directory: /home/glue_user/.cache/pip/wheels/ad/fa/e3/0ed85d0d18ebd9e1344ffd6c1d1d0031af21eee1988ba36050
Successfully built pyspark



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import json
from pyspark.sql import types
from delta import *
from pyspark.sql.functions import *
from delta.tables import *
from pyspark.sql.functions import *
from datetime import datetime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load dos arquivos raw data local para ingestao em delta tables

In [5]:
%%sh
ls ../../data_set

characters.csv
characters_To_Comics.csv
charcters_stats.csv
comics.csv
delta
marvel_characters_info.csv
marvel_dc_characters.csv
marvel_dc_characters.xlsx
superheroes_power_matrix.csv


In [6]:
nam_path='/home/glue_user/workspace/jupyter_workspace/data_set'
nam_file='characters.csv'
bol_header=True
json_path='/home/glue_user/workspace/jupyter_workspace/src/bronze_data/schemas/characters.json'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
with open(json_path,'r') as f:
    data = json.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
schemaFromJson = StructType.fromJson(data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df=spark.read.csv(f'{nam_path}/{nam_file}', header=True, schema=schemaFromJson)
desc_date=datetime.now()
desc_date=desc_date.strftime('%Y%m%d')
df=df.withColumn('dat_load', lit(desc_date))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------+
|characterID|                name|dat_load|
+-----------+--------------------+--------+
|    1009220|     Captain America|20230418|
|    1010740|      Winter Soldier|20230418|
|    1009471|           Nick Fury|20230418|
|    1009552|        S.H.I.E.L.D.|20230418|
|    1009228|       Sharon Carter|20230418|
|    1011109|    X-Men (Ultimate)|20230418|
|    1010808|Hawkeye (Kate Bis...|20230418|
|    1009515|            Punisher|20230418|
|    1009252|          Crossbones|20230418|
|    1009535|           Red Skull|20230418|
+-----------+--------------------+--------+
only showing top 10 rows

### Salvando arquivo no Bucket S3

In [13]:
nam_bucket='s3://marvel-bronze-dev'
nam_table='characters'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.write.format("delta").mode("overwrite").partitionBy('dat_load').save(f'{nam_bucket}/{nam_table}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Verificando se existe os dados para carga de update

In [11]:
import boto3

s3 = boto3.client('s3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
response = s3.list_objects_v2(Bucket=nam_bucket.replace('s3://',''), Prefix=nam_table)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
response['KeyCount']>1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [52]:
if response['KeyCount']>1:
    
else:
    df.write.format("delta").partitionBy('dat_load').save(f'{nam_bucket}/{nam_table}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

In [17]:
%%sh
pwd

/home/glue_user/workspace/jupyter_workspace/src/bronze_data


In [18]:
table = DeltaTable.forPath(spark, f'{nam_bucket}/{nam_table}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
table.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DeltaTable' object has no attribute 'show'
Traceback (most recent call last):
AttributeError: 'DeltaTable' object has no attribute 'show'



In [20]:
df = table.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------+
|characterID|                name|dat_load|
+-----------+--------------------+--------+
|    1009220|     Captain America|20230418|
|    1010740|      Winter Soldier|20230418|
|    1009471|           Nick Fury|20230418|
|    1009552|        S.H.I.E.L.D.|20230418|
|    1009228|       Sharon Carter|20230418|
|    1011109|    X-Men (Ultimate)|20230418|
|    1010808|Hawkeye (Kate Bis...|20230418|
|    1009515|            Punisher|20230418|
|    1009252|          Crossbones|20230418|
|    1009535|           Red Skull|20230418|
+-----------+--------------------+--------+
only showing top 10 rows

In [22]:
df = spark.read.format("delta").load(f'{nam_bucket}/{nam_table}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------+
|characterID|                name|dat_load|
+-----------+--------------------+--------+
|    1009220|     Captain America|20230418|
|    1010740|      Winter Soldier|20230418|
|    1009471|           Nick Fury|20230418|
|    1009552|        S.H.I.E.L.D.|20230418|
|    1009228|       Sharon Carter|20230418|
|    1011109|    X-Men (Ultimate)|20230418|
|    1010808|Hawkeye (Kate Bis...|20230418|
|    1009515|            Punisher|20230418|
|    1009252|          Crossbones|20230418|
|    1009535|           Red Skull|20230418|
+-----------+--------------------+--------+
only showing top 10 rows

In [33]:
    files = spark.format('delta').sparkContext.(f's3a://{bucket_name}/{file_name}')
    if files.isEmpty():
        print(f'O arquivo {file_name} não existe no bucket {bucket_name}.')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    files = spark.format('delta').sparkContext.(f's3a://{bucket_name}/{file_name}')
                                               ^
SyntaxError: invalid syntax

